In [ ]:
pip install openai

In [3]:
import numpy as np
import pandas as pd
import os
import sys
from sentence_transformers import SentenceTransformer, util
import torch
import time
import json


In [6]:
import random
from google.colab import files

input_file = 'hugging_face_train.csv'
train_output_file = '3.5_train.csv'
test_output_file = '3.5_test.csv'

# Step 1: Read all lines from the input file
with open(input_file, 'r') as infile:
    lines = infile.readlines()

# Step 2: Randomly select 50,000 lines for training
train_lines = random.sample(lines, 50000)

# Step 3: Remove the selected training lines from the list of all lines
remaining_lines = list(set(lines) - set(train_lines))

# Step 4: Randomly select 2,500 lines for testing
test_lines = random.sample(remaining_lines, 2500)

# Step 5: Write the selected lines to their respective output files
with open(train_output_file, 'w') as train_file:
    train_file.writelines(train_lines)

with open(test_output_file, 'w') as test_file:
    test_file.writelines(test_lines)

print(f"Random 50,000 lines written to {train_output_file}")
print(f"Random 2,500 lines written to {test_output_file}")

# Step 6: Download the files to your local machine
files.download(train_output_file)
files.download(test_output_file)



Random 50,000 lines written to 3.5_train.csv
Random 2,500 lines written to 3.5_test.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [1]:
!pip install openai

import openai

In [4]:
!pip install --upgrade openai

In [6]:
openai.api_key = 'sk-proj-8oLvnNGJLnlgW4SQOoHwT3BlbkFJ8c24SWE59CoO4sTxlDC7'

In [ ]:
# Begin the training


def upload_file(file_path):
    with open(file_path, 'rb') as f:
        response = openai.File.create(
            file=f,
            purpose='fine-tune'
        )
    return response['id']

dataset_path = 'gpt_3.5_train.txt'
file_id = upload_file(dataset_path)
print(f"Uploaded file ID: {file_id}")

def create_fine_tune_job(file_id, model='gpt-3.5-turbo'):
    response = openai.FineTune.create(
        training_file=file_id,
        model=model
    )
    return response

fine_tune_response = create_fine_tune_job(file_id)
print(f"Fine-tune job created: {fine_tune_response['id']}")

import time

def check_fine_tune_status(job_id):
    response = openai.FineTune.retrieve(id=job_id)
    status = response['status']
    return status

job_id = fine_tune_response['id']
print("Monitoring fine-tuning job...")
while True:
    status = check_fine_tune_status(job_id)
    print(f"Job status: {status}")
    if status == 'succeeded':
        print("Fine-tuning job completed successfully!")
        break
    elif status == 'failed':
        print("Fine-tuning job failed.")
        break
    time.sleep(60)

fine_tuned_model = openai.FineTune.retrieve(id=job_id)['fine_tuned_model']
print(f"Fine-tuned model name: {fine_tuned_model}")

with open('fine_tuned_model_name.txt', 'w') as f:
    f.write(fine_tuned_model)

print("done")





In [13]:
!pip install --upgrade openai


  Using cached openai-1.30.4-py3-none-any.whl (320 kB)
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0


In [2]:
!pip install openai==0.28
import openai

In [10]:
import time

In [3]:
import json

input_file = 'gpt_3.5_train.txt'
output_file = 'formatted_dataset.jsonl'

# Step 1: Read the input file
with open(input_file, 'r') as infile:
    lines = infile.readlines()

# Step 2: Format each line into a JSON object
json_lines = []
for line in lines:
    # Split the line into clue, length, and answer
    parts = line.strip().split(', ')
    if len(parts) == 3:
        clue, length, answer = parts
        # Create a JSON object
        json_obj = {
            "prompt": f"{clue}, {length}",
            "completion": answer.strip()
        }
        json_lines.append(json_obj)

# Step 3: Write the JSON objects to a JSONL file
with open(output_file, 'w') as outfile:
    for json_obj in json_lines:
        outfile.write(json.dumps(json_obj) + '\n')

print(f"Converted {len(lines)} lines to JSONL format and saved to {output_file}")

from google.colab import files
files.download(output_file)


Converted 50000 lines to JSONL format and saved to formatted_dataset.jsonl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:
import json

# Check the JSONL file format
output_file = 'formatted_dataset.jsonl'
with open(output_file, 'r') as infile:
    for line in infile:
        try:
            json_obj = json.loads(line)
            if 'prompt' not in json_obj or 'completion' not in json_obj:
                print(f"Invalid format: {line}")
        except json.JSONDecodeError as e:
            print(f"JSON decode error: {line}")
print("All good")

All good


In [1]:
!pip install openai
import openai
import time
import pandas as pd
import numpy as np


In [2]:
pip install openai==0.28

In [10]:
import json

input_file = 'gpt_3.5_train.txt'
output_file = 'chat_formatted_dataset.jsonl'

# Read the input file
with open(input_file, 'r') as infile:
    lines = infile.readlines()

# Format each line into a JSON object for chat
json_lines = []
for line in lines:
    parts = line.strip().split(', ')
    if len(parts) == 3:
        clue, length, answer = parts
        json_obj = {
            "messages": [
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"{clue}, {length}"},
                {"role": "assistant", "content": answer.strip()}
            ]
        }
        json_lines.append(json_obj)

# Write the JSON objects to a JSONL file
with open(output_file, 'w') as outfile:
    for json_obj in json_lines:
        outfile.write(json.dumps(json_obj) + '\n')

print(f"Converted {len(lines)} lines to chat format and saved to {output_file}")

# Download the file to your local machine
from google.colab import files
files.download(output_file)


Converted 50000 lines to chat format and saved to chat_formatted_dataset.jsonl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
openai.api_key = 'sk-proj-8oLvnNGJLnlgW4SQOoHwT3BlbkFJ8c24SWE59CoO4sTxlDC7'

In [6]:
import json
import random

input_file = 'chat_formatted_dataset.jsonl'
output_file = 'chat_formatted_dataset_2000.jsonl'

# Read the input JSONL file
with open(input_file, 'r') as infile:
    lines = infile.readlines()

# Ensure there are at least 2000 lines
if len(lines) < 2000:
    raise ValueError("The input file contains fewer than 2000 lines.")

# Randomly select 2000 lines
selected_lines = random.sample(lines, 2000)

# Write the selected lines to the output JSONL file
with open(output_file, 'w') as outfile:
    for line in selected_lines:
        outfile.write(line)

print(f"Selected 2000 lines and saved to {output_file}")

# Download the file to your local machine
from google.colab import files
files.download(output_file)


Selected 2000 lines and saved to chat_formatted_dataset_2000.jsonl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
train_output_file = 'chat_formatted_dataset_2000.jsonl'

# Step 2: Upload the dataset
def upload_file(file_path):
    response = openai.File.create(
        file=open(file_path, "rb"),
        purpose='fine-tune'
    )
    return response['id']

train_file_id = upload_file(train_output_file)
print(f"Uploaded train file ID: {train_file_id}")

# Step 3: Create a fine-tuning job
def create_fine_tune_job(training_file_id, model='gpt-3.5-turbo'):
    response = openai.FineTune.create(
        training_file=training_file_id,
        model=model
    )
    return response

# Use the correct endpoint for creating fine-tunes
fine_tune_response = openai.FineTuningJob.create(training_file=train_file_id, model='gpt-3.5-turbo')
print(f"Fine-tune job created: {fine_tune_response['id']}")

def get_fine_tune_job_details(job_id):
    response = openai.FineTuningJob.retrieve(id=job_id)
    return response


# Step 4: Monitor the fine-tuning job
def check_fine_tune_status(job_id):
    response = openai.FineTuningJob.retrieve(id=job_id)
    status = response['status']
    return status

job_id = fine_tune_response['id']
print(job_id)

print("Monitoring fine-tuning job...")
while True:
    status = check_fine_tune_status(job_id)
    #details = get_fine_tune_job_details(job_id)
    #print(f"Details: {details}")
    print(f"Job status: {status}")
    if status == 'succeeded':
        print("Fine-tuning job completed successfully!")
        break
    elif status == 'failed':
        print("Fine-tuning job failed.")
        break
    time.sleep(45)

# Step 5: Save the fine-tuned model name
fine_tuned_model = openai.FineTuningJob.retrieve(id=job_id)['fine_tuned_model']
print(f"Fine-tuned model name: {fine_tuned_model}")

# Save the fine-tuned model name to a file
with open('fine_tuned_model_name.txt', 'w') as f:
    f.write(fine_tuned_model)

Uploaded train file ID: file-crnSMWCLbtyChDJJW1Mha3uq
Fine-tune job created: ftjob-z57JP0WeZUuwIkTShHCR5qWF
ftjob-z57JP0WeZUuwIkTShHCR5qWF
Monitoring fine-tuning job...
Job status: validating_files
Job status: validating_files
Job status: validating_files
Job status: validating_files
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: run

In [1]:
!pip install openai
import openai

openai.api_key = 'sk-proj-8oLvnNGJLnlgW4SQOoHwT3BlbkFJ8c24SWE59CoO4sTxlDC7'

In [2]:
pip install openai==0.28

In [ ]:
import openai

# Step 3: Load the fine-tuned model name from the file
with open('fine_tuned_model_name.txt', 'r') as f:
    fine_tuned_model = f.read().strip()

In [25]:
# Step 4: Generate a completion using the fine-tuned model
def generate_completion(prompt, model):
    response = openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=50
    )
    return response.choices[0].message['content'].strip()


def generate_multiple_completions(prompt, model, num_completions=5, max_tokens=50):
    responses = openai.ChatCompletion.create(
        model=model,
        prompt=prompt,
        max_tokens=max_tokens,
        n=num_completions,
        stop=None,
        temperature=0.7
    )
    completions = [response['text'].strip() for response in responses['choices']]
    return completions

prompt = "Fennel or sweet cicely"
completions = generate_multiple_completions(prompt, fine_tuned_model, num_completions=5)
for i, completion in enumerate(completions, 1):
    print(f"Completion {i}: {completion}")





# Use the fine-tuned model
print(f"Using fine-tuned model: {fine_tuned_model}")

prompt = "Some crumbly blocks, 4"
completion = generate_completion(prompt, fine_tuned_model)
print(f"Prompt: {prompt}")
print(f"Completion: {completion}")


InvalidRequestError: Missing required parameter: 'messages'.

In [24]:
import openai

def generate_unique_completions(prompt, model, num_completions=5, max_tokens=50):
    response = openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=max_tokens,
        n=num_completions * 2,  # Request more completions to increase chances of uniqueness
        stop=None,
        temperature=0.9,  # Increase temperature for more diverse answers
        top_p=0.9  # Use top-p sampling for diversity
    )
    completions = set()
    for choice in response['choices']:
        completions.add(choice['message']['content'].strip())
        if len(completions) >= num_completions:
            break

    return list(completions)[:num_completions]  # Return only the requested number of unique completions

# Example usage
prompt = "Tries for a role, 5, R___S"
completions = generate_unique_completions(prompt, fine_tuned_model, num_completions=5)
for i, completion in enumerate(completions, 1):
    print(f"Completion {i}: {completion}")


Completion 1: READS:^{
Completion 2: READS|--------------------------------------------------------------------------
Completion 3: READS
Completion 4: AUDITS
Completion 5: READSF


In [ ]:
!pip install openai==0.28
import openai

In [6]:

openai.api_key = 'sk-proj-04xTZxjLUmsxc05S1jlNT3BlbkFJ1pYCgmK5xeM0vB9YqlJR'


def upload_file(file_path):
    with open(file_path, 'rb') as f:
        response = openai.File.create(
            file=f,
            purpose='fine-tune'
        )
    return response['id']

dataset_path = 'gpt_3.5_train.txt'
file_id = upload_file(dataset_path)
print(f"Uploaded file ID: {file_id}")

# Step 3: Create a fine-tuning job
def create_fine_tune_job(file_id, model='gpt-3.5-turbo'):
    response = openai.FineTune.create(
        training_file=file_id,
        model=model
    )
    return response

fine_tune_response = create_fine_tune_job(train_file_id)
print(f"Fine-tune job created: {fine_tune_response['id']}")

# Step 4: Monitor the fine-tuning job
def check_fine_tune_status(job_id):
    response = openai.FineTune.retrieve(id=job_id)
    status = response['status']
    return status

job_id = fine_tune_response['id']

print("Monitoring fine-tuning job...")
while True:
    status = check_fine_tune_status(job_id)
    print(f"Job status: {status}")
    if status == 'succeeded':
        print("Fine-tuning job completed successfully!")
        break
    elif status == 'failed':
        print("Fine-tuning job failed.")
        break
    time.sleep(60)  # Check status every minute

# Step 5: Save the fine-tuned model name
fine_tuned_model = openai.FineTune.retrieve(id=job_id)['fine_tuned_model']
print(f"Fine-tuned model name: {fine_tuned_model}")

# Save the fine-tuned model name to a file
with open('fine_tuned_model_name.txt', 'w') as f:
    f.write(fine_tuned_model)

Uploaded file ID: file-tXI1N4xwQgDm6wDJZAE6TQie


NameError: name 'train_file_id' is not defined

In [ ]:
# Potential Load???? Figure this out?

# Step 1: Install the OpenAI library
!pip install openai

# Step 2: Import required libraries and set up the OpenAI API key
import openai

# Set your OpenAI API key
openai.api_key = 'YOUR_API_KEY'

# Step 3: Load the fine-tuned model name from the file
with open('fine_tuned_model_name.txt', 'r') as f:
    fine_tuned_model = f.read().strip()

# Step 4: Generate a completion using the fine-tuned model
def generate_completion(prompt, model):
    response = openai.Completion.create(
        model=model,
        prompt=prompt,
        max_tokens=50
    )
    return response.choices[0].text.strip()

# Use the fine-tuned model
print(f"Using fine-tuned model: {fine_tuned_model}")

prompt = "Fennel or sweet cicely"
completion = generate_completion(prompt, fine_tuned_model)
print(f"Prompt: {prompt}")
print(f"Completion: {completion}")


In [ ]:
def generate_multiple_completions(prompt, model, num_completions=5, max_tokens=50):
    responses = openai.Completion.create(
        model=model,
        prompt=prompt,
        max_tokens=max_tokens,
        n=num_completions,
        stop=None,
        temperature=0.7
    )
    completions = [response['text'].strip() for response in responses['choices']]
    return completions

prompt = "Fennel or sweet cicely"
completions = generate_multiple_completions(prompt, fine_tuned_model, num_completions=5)
for i, completion in enumerate(completions, 1):
    print(f"Completion {i}: {completion}")



In [7]:
import random
from google.colab import files

input_file = 'reformatted.txt'
train_output_file = 'gpt_3.5_train.txt'
test_output_file = 'gpt_3.5_test.txt'

# Step 1: Read all lines from the input file
with open(input_file, 'r') as infile:
    lines = infile.readlines()

# Step 2: Randomly select 50,000 lines for training
train_lines = random.sample(lines, 50000)

# Step 3: Remove the selected training lines from the list of all lines
remaining_lines = list(set(lines) - set(train_lines))

# Step 4: Randomly select 2,500 lines for testing
test_lines = random.sample(remaining_lines, 2500)

# Step 5: Write the selected lines to their respective output files
with open(train_output_file, 'w') as train_file:
    train_file.writelines(train_lines)

with open(test_output_file, 'w') as test_file:
    test_file.writelines(test_lines)

print(f"Random 50,000 lines written to {train_output_file}")
print(f"Random 2,500 lines written to {test_output_file}")

# Step 6: Download the files to your local machine
files.download(train_output_file)
files.download(test_output_file)

Random 50,000 lines written to gpt_3.5_train.txt
Random 2,500 lines written to gpt_3.5_test.txt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [1]:
# Potential Loopy Belief Code

import numpy as np

# Example nested dictionary input
clues = {
    "clue1": {"SKY": 0.2, "FLY": 0.5, "HEY": 0.3},
    "clue2": {"HI": 0.4, "OH": 0.6},
    # Add more clues
}

# Example grid format (coordinates for each clue)
grid = {
    "clue1": [(0, 0), (0, 1), (0, 2)],
    "clue2": [(0, 2), (1, 2)],
    # Add more clues with their respective coordinates
}

# Initialize messages
messages = {clue: {neighbor: np.ones(len(possible_answers)) / len(possible_answers)
                   for neighbor in grid if neighbor != clue}
            for clue, possible_answers in clues.items()}

# Update messages (loopy belief propagation)
def update_messages(clues, grid, messages, iterations=10):
    for _ in range(iterations):
        new_messages = messages.copy()
        for clue, neighbors in grid.items():
            for neighbor in neighbors:
                for possible_answer, prob in clues[clue].items():
                    message_product = 1
                    for other_neighbor in grid[clue]:
                        if other_neighbor != neighbor:
                            message_product *= messages[other_neighbor][clue][possible_answer]
                    new_messages[clue][neighbor][possible_answer] = prob * message_product
                new_messages[clue][neighbor] /= new_messages[clue][neighbor].sum()
        messages = new_messages
    return messages

# Calculate beliefs
def calculate_beliefs(clues, grid, messages):
    beliefs = {}
    for clue, possible_answers in clues.items():
        belief = np.ones(len(possible_answers))
        for neighbor in grid[clue]:
            belief *= messages[neighbor][clue]
        belief *= np.array(list(possible_answers.values()))
        beliefs[clue] = belief / belief.sum()
    return beliefs

# Run loopy belief propagation
messages = update_messages(clues, grid, messages)
beliefs = calculate_beliefs(clues, grid, messages)

# Determine final answers
final_answers = {clue: max(possible_answers, key=lambda ans: beliefs[clue][ans_idx])
                 for clue, possible_answers in clues.items()
                 for ans_idx, ans in enumerate(possible_answers)}

print("Final Answers:", final_answers)


KeyError: (0, 1)

In [3]:
import numpy as np

# Example nested dictionary input
clues = {
    "clue1": {"SKY": 0.2, "FLY": 0.5, "BRO": 0.3},
    "clue2": {"HI": 0.4, "OH": 0.6},
    # Add more clues
}

# Example grid format (coordinates for each clue)
grid = {
    "clue1": [(0, 0), (0, 1), (0, 2)],
    "clue2": [(0, 2), (1, 2)],
    # Add more clues with their respective coordinates
}

# Find neighbors based on grid intersections
def find_neighbors(grid):
    coord_to_clues = {}
    for clue, coords in grid.items():
        for coord in coords:
            if coord not in coord_to_clues:
                coord_to_clues[coord] = []
            coord_to_clues[coord].append(clue)

    neighbors = {clue: set() for clue in grid}
    for coord, clues_at_coord in coord_to_clues.items():
        for clue in clues_at_coord:
            neighbors[clue].update(clues_at_coord)
            neighbors[clue].remove(clue)

    return neighbors

# Get neighbors for each clue
neighbors = find_neighbors(grid)

# Initialize messages
messages = {clue: {neighbor: np.ones(len(clues[neighbor])) / len(clues[neighbor])
                   for neighbor in neighbors[clue]}
            for clue in clues}

# Print initialized messages
print("Initialized messages:")
for clue, msgs in messages.items():
    print(f"{clue}:")
    for neighbor, msg in msgs.items():
        print(f"  to {neighbor}: {msg}")

# Example function to update messages (simplified for illustration)
def update_messages(clues, neighbors, messages, iterations=10):
    for _ in range(iterations):
        new_messages = {clue: {neighbor: np.ones(len(clues[neighbor])) / len(clues[neighbor])
                               for neighbor in neighbors[clue]}
                        for clue in clues}
        for clue in clues:
            for neighbor in neighbors[clue]:
                for possible_answer, prob in clues[clue].items():
                    message_product = 1
                    for other_neighbor in neighbors[clue]:
                        if other_neighbor != neighbor:
                            message_product *= messages[other_neighbor][clue][possible_answer]
                    new_messages[clue][neighbor][possible_answer] = prob * message_product
                new_messages[clue][neighbor] /= new_messages[clue][neighbor].sum()
        messages = new_messages
    return messages

# Run loopy belief propagation
messages = update_messages(clues, neighbors, messages)

# Example function to calculate beliefs
def calculate_beliefs(clues, neighbors, messages):
    beliefs = {}
    for clue, possible_answers in clues.items():
        belief = np.ones(len(possible_answers))
        for neighbor in neighbors[clue]:
            belief *= messages[neighbor][clue]
        belief *= np.array(list(possible_answers.values()))
        beliefs[clue] = belief / belief.sum()
    return beliefs

# Calculate beliefs
beliefs = calculate_beliefs(clues, neighbors, messages)

# Determine final answers
final_answers = {clue: max(clues[clue], key=lambda ans: beliefs[clue][list(clues[clue].keys()).index(ans)])
                 for clue in clues}

print("Final Answers:", final_answers)


Initialized messages:
clue1:
  to clue2: [0.5 0.5]
clue2:
  to clue1: [0.33333333 0.33333333 0.33333333]


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices